In [1]:
# import all libraries
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf

In [2]:
from tqdm import tqdm
# this will add progress bar to "for loop"

In [3]:
#shows all files in dataset
files=os.listdir("C:/Users/sanji/Documents/dataset/")
files

['aishwarya_rai',
 'angelina_jolie',
 'arnold_schwarzenegger',
 'bhuvan_bam',
 'brad_pitt',
 'Courteney_Cox',
 'David_Schwimmer',
 'dhoni',
 'hardik_pandya',
 'Jennifer_Aniston',
 'Jim_Parsons',
 'Johnny_Galeck',
 'Kunal_Nayya',
 'Lisa_Kudrow',
 'manoj_bajpayee',
 'Matthew_Perry',
 'Matt_LeBlanc',
 'messi',
 'mohamed_ali',
 'Pankaj_Tripathi',
 'pewdiepie',
 'random_person',
 'ROHIT_SHARMA',
 'ronaldo',
 'Sachin_Tendulka',
 'scarlett_johansson',
 'Simon_Helberg',
 'suresh_raina',
 'sylvester_stallone',
 'virat_kohli']

In [4]:
# We will now collect image array and label array
image_array=[]
# list we will convert into an array
label_array=[] 
path = "C:/Users/sanji/Documents/dataset/"
# loop through each dataset
for i in range(len(files)):
    # list of all files in each face folder
    # path=path+files[i] -> complete path
    file_sub=os.listdir(path+files[i])
    for k in tqdm(range(len(file_sub))):
        # loop through each faces
        try:
            img = cv2.imread(path+files[i]+"/"+file_sub[k])
            # path+files[i]+"/"+file_sub[k] will look like this
            
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
            # convert image color from BGR to RGB
            # resize the image to (96, 96)
            img = cv2.resize(img, (96, 96))
            # if you want to train on different size change this
            # training on small size like (48, 48) will increase frame rate in app
            # training on large size like (192, 192) will increase accuracy but will 
            # decrease frame rate in app
            image_array.append(img)
            # add it to image_array
            label_array.append(i)
            # i is changing from 0-29
            # so we will use it as our label
        except:
            pass

100%|██████████| 391/391 [00:03<00:00, 114.42it/s]


In [5]:
# now we will clear our ram memory
import gc
gc.collect()

73

In [6]:
# now we will convert image_array list to array
# we also divide image by 255 to scale image from 0-255 to 0-1
image_array = np.array(image_array)/255.0
label_array = np.array(label_array)

In [7]:
image_array[0]
#  0-255 convert to 0-1

array([[[0.45490196, 0.38823529, 0.38039216],
        [0.25490196, 0.18823529, 0.18039216],
        [0.38823529, 0.33333333, 0.32156863],
        ...,
        [0.52941176, 0.5254902 , 0.50980392],
        [0.27058824, 0.28235294, 0.2627451 ],
        [0.25098039, 0.25098039, 0.24705882]],

       [[0.52941176, 0.45490196, 0.43529412],
        [0.4       , 0.3254902 , 0.30588235],
        [0.45098039, 0.37647059, 0.36078431],
        ...,
        [0.30588235, 0.30588235, 0.29803922],
        [0.30980392, 0.31764706, 0.31372549],
        [0.25098039, 0.24705882, 0.26666667]],

       [[0.44705882, 0.36862745, 0.34117647],
        [0.40784314, 0.31764706, 0.28627451],
        [0.47843137, 0.38431373, 0.36862745],
        ...,
        [0.36862745, 0.36078431, 0.36470588],
        [0.29411765, 0.29803922, 0.30980392],
        [0.24313725, 0.23921569, 0.26666667]],

       ...,

       [[0.01176471, 0.01176471, 0.01176471],
        [0.01176471, 0.01176471, 0.01176471],
        [0.00784314, 0

In [8]:
# now we will split and shuffle our dataset for trainig and validation
from sklearn.model_selection import train_test_split
#                                                   images       label        splitting ratio 
X_train, X_test, Y_train, Y_test = train_test_split(image_array, label_array, test_size=0.15, random_state=0)

In [9]:
# X_train and Y_train are 85% of images and label
# X_test and Y_test are 15% of images and label

In [10]:
# now we will create our model
# we will use image classification model
from keras import layers,callbacks,utils, applications, optimizers
from keras import Sequential, Model

In [11]:
# create model
model = Sequential()
# This time I will use EfficientNet pretrained model
# you have several options in EfficientNet B0 to B7
# B0 will have high frame rate but with low accuracy
# B7 will have low frame rate but high accuracy
pretrained_model = tf.keras.applications.EfficientNetB0(input_shape = (96,96,3), 
                                                        include_top = False,
                                                        weights="imagenet")
# add it to our sequential model
model.add(pretrained_model)
# add some layers to improve accuracy
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.3))

# add dense layer with number of output
model.add(layers.Dense(1))
# output is 1 value which changes from 0 to 29
# You can add some more layer to increase accuracy
# but we will use basic model 
# Run
# show summary 

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 3, 3, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 4,008,829 (15.29 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [13]:
# now compile model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# optimizer: You can change it to some other optimizer to improve accuracy(SGD,etc)
# loss: Try different losses to improve accuracy


In [14]:
# create a checkpoint to save the best accuracy model
ckp_path = "trained_model/model.weights.h5"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,
                                                      monitor='val_loss',
                                                      mode="auto",
                                                      save_best_only=True,
                                                      save_weights_only=True)

# monitor="val_mae": watch mae of validation set & when it decreases save the model 
# mode: it is use to check for decrease of increase in mae
# mode:(auto, min, max)
# save_weights_only: if true save only weight (matrix of number)


In [15]:
# create a lr reducer which will decrease learning rate when accuracy doesn't
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factors=0.9,
                                                monitor="val_mae",
                                                mode="auto",
                                                cooldown=0,
                                                patience=5,
                                                verbose=1,
                                                min_lr=1e-6)
# patience: wait for 5 epoch then decrease learning rate
# verbose : show accuracy every epoch
# min_lr=minimum learning rate

# you can choose other reduce_lr to increase accuracy

In [ ]:
# train model
Epoch=300
Batch_Size=64
history=model.fit(X_train,Y_train,
                  validation_data=(X_test, Y_test),
                  batch_size=Batch_Size,
                  epochs=Epoch,
                  callbacks=[model_checkpoint, reduce_lr])

Epoch 1/300


In [ ]:
# see val_mae is decreasing

In [ ]:
# after the training is finished
# load best model
model.load_weights(ckp_path)

In [ ]:
# convert best model to tensorflow lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()
# save model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
# if you want to see prediction of validation set
prediction_val = model.predict(X_test, batch_size=64)
prediction_val[:20] # show first 20 value

In [ ]:
# original label
X_test[:20]
# Display already trained model

In [ ]:
# to save model
# this will run in fresh notebook